In [1]:
#Imports y herramientas varias

import matplotlib.pyplot as plt
import numpy as np
import pandas as pnd
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import pydotplus
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor

In [2]:
#Importar datos originales
path = "Datos/"

clientes = pnd.read_excel(path + "CREDITRISK_RAW.xlsx")

In [8]:
#Exploración de atributos
for key in clientes.keys():
    print "Atributo:", key
    print "\tTipo:", clientes[key].dtype

Atributo: ID
	Tipo: int64
Atributo: GENERO
	Tipo: object
Atributo: RENTA
	Tipo: float64
Atributo: EDAD
	Tipo: float64
Atributo: NIV_EDUC
	Tipo: object
Atributo: E_CIVIL
	Tipo: object
Atributo: COD_OFI
	Tipo: int64
Atributo: COD_COM
	Tipo: object
Atributo: CIUDAD
	Tipo: object
Atributo: Crédito_1
	Tipo: int64
Atributo: Crédito_2
	Tipo: int64
Atributo: Crédito_3
	Tipo: float64
Atributo: Crédito_4
	Tipo: int64
Atributo: Monto solicitado
	Tipo: float64
Atributo: Días de Mora
	Tipo: int64
Atributo: Monto Deuda Promedio
	Tipo: int64
Atributo: Número de meses inactivo
	Tipo: int64
Atributo: numero de cuotas
	Tipo: int64
Atributo: Aval
	Tipo: object
Atributo: PAGA
	Tipo: object


In [4]:
#Extracción de clientes con algún dato en blanco
empty_attribute_clients = clientes[clientes.isnull().any(axis=1)]
clean_clients = clientes[~clientes.isnull().any(axis=1)]

In [5]:
print len(clientes)
print len(clean_clients)
print len(empty_attribute_clients)

2294
2240
54


In [11]:
llaves = clean_clients.keys()

for j in range(len(llaves)):
    if clean_clients[llaves[j]].dtype == 'int64' or clean_clients[llaves[j]].dtype == 'float64':
        continue
    if llaves[j] in ['id']:
        continue
    values = clean_clients[llaves[j]].unique()
    dic = {}
    for i in range(len(values)):
        dic[values[i]] = i

    clean_clients[llaves[j]] = clean_clients[llaves[j]].apply(lambda x: dic[x])

#Paga: 0, No Paga: 1

/Users/synaptic/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [27]:
clean_clients

,ID,GENERO,RENTA,EDAD,NIV_EDUC,E_CIVIL,COD_OFI,COD_COM,CIUDAD,Crédito_1,Crédito_2,Crédito_3,Crédito_4,Monto solicitado,Días de Mora,Monto Deuda Promedio,Número de meses inactivo,numero de cuotas,Aval,PAGA
0,1,0,463000.0,55.0,0,0,27,0,0,0,-4030,0.0,292673,142.0,0,81,1,18,0,0
1,2,0,1200000.0,59.0,0,0,121,0,0,0,0,27797549.0,0,16.0,0,34519,1,60,0,1
2,3,0,526000.0,49.0,1,1,59,1,1,0,750000,0.0,0,0.0,0,8016,0,14,0,1
3,4,1,1500000.0,47.0,2,1,138,2,2,0,976444,0.0,0,13.0,10,25323,1,66,0,0
4,5,1,2600000.0,41.0,2,0,60,3,3,0,115440,0.0,4500000,7960.0,14,4566,3,77,1,1
5,6,0,1250000.0,30.0,0,2,63,4,4,0,799000,0.0,0,147.0,0,0,1,55,0,1
6,7,1,724000.0,47.0,2,0,138,2,2,0,500000,0.0,1963,0.0,0,2,1,24,0,0
7,8,1,1196000.0,53.0,2,0,27,0,0,0,0,0.0,0,0.0,4,0,1,60,0,0
8,9,0,510000.0,39.0,0,0,96,5,5,0,1050000,0.0,279914,305.0,0,0,1,10,0,1
9,10,1,685000.0,51.0,2,0,82,6,1,0,164272,0.0,99401,114.0,0,1152,1,26,0,0
